In [10]:
#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from  nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
from nltk import classify
from nltk.tokenize import TweetTokenizer
from nltk import NaiveBayesClassifier
from nltk.tag import pos_tag


#import regex
import re

#import BeautifulSoup
from bs4 import BeautifulSoup

# Scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import text 
from sklearn.decomposition import LatentDirichletAllocation
from nltk.util import ngrams
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer 


# Word2vec
import gensim

#plotting
import matplotlib.pyplot as plt

# etc
import string
import pandas as pd
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools


In [2]:
Columns  = ["sentiment", "user_id", "date", "query", "username", "text"]
Date = "ISO-8859-1"

In [3]:
tweets = pd.read_csv("Data/tweets.csv", encoding = Date, names = Columns)

In [4]:
tweets =tweets[['sentiment', 'text']]

In [7]:
twit_samp = tweets.sample(n=10000,replace=False)

In [9]:
#Tokenize

tk = TweetTokenizer(reduce_len=True)

lst = []

# Separating our features (text) and our labels into two lists to smoothen our work
X = twit_samp['text'].tolist()
y = twit_samp['sentiment'].tolist()

# Building our data list, that is a list of tuples, where each tuple is a pair of the tokenized text
# and its corresponding label
for x, y in zip(X, y):
    if y == 4:
        lst.append((tk.tokenize(x), 1))
    else:
        lst.append((tk.tokenize(x), 0))
        

In [14]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lem_sent = []
    for word, tag in pos_tag(tokens):
        # convert the pos_tag output tags to a tag format that the WordNetLemmatizer can interpret
        # typicallyif a tag starts with NN, the word is a noun and if it stars with VB, the word is a verb.
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lem_sent.append(lemmatizer.lemmatize(word, pos))
    return lem_sent

In [48]:
def clean(tokens):

    if token == 'amp' or token == 'quot' or token == 'got' or token == 'go':
        return ''
    if token == '4got' or token == '4gotten':
        return 'forget'
  
    return token

In [69]:
def lem_stem(tokens):

    new_token = []
 
    #mentions = "(@[A-Za-z0-9_]+)"
    for token, tag in pos_tag(tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
      #  token = re.sub(mentions,"", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
    #    stemmer = PorterStemmer()
        token = lemmatizer.lemmatize(token, pos)
      #  token = stemmer.stem(token)
        
        new_token = (token.lower())
        

In [70]:
new_l = []
for tokens, label in lst:
    new_l.append((lem_stem(tokens), label))


In [72]:
new_l

[(None, 1),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 1),
 (None, 1),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 0),
 (None, 0),
 (None, 1),
 (None, 1),
 (No

In [19]:
stemmer = PorterStemmer()
def clean(text, stem=True):
    #removed links and chars
    remove = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text = re.sub(remove, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
        
            else:
                tokens.append(token)
    return " ".join(tokens)

AttributeError: 'tuple' object has no attribute 'isdigit'

In [ ]:
def process_word(word, pos):
    pos_dict = {'J':'a', 'V':'v', 'N':'n', 'R': 'r'}
    if pos[0] in pos_dict:
        return lemmatizer.lemmatize(word.lower(), pos_dict[pos[0]])
    else:
        return lemmatizer.lemmatize(word.lower())